get gemini llm to generate some data

In [ ]:
from llm import get_gemini_llm

llm = get_gemini_llm()

In [ ]:
response = llm.generate_content("Liste 10 romanciers français célébres.")
print(response.text)

Voici 10 romanciers français célèbres :

1. **Victor Hugo:** ( *Les Misérables*, *Notre-Dame de Paris*)
2. **Alexandre Dumas:** ( *Les Trois Mousquetaires*, *Le Comte de Monte-Cristo*)
3. **Marcel Proust:** (*À la recherche du temps perdu*)
4. **Albert Camus:** (*L'Étranger*, *La Peste*)
5. **Antoine de Saint-Exupéry:** (*Le Petit Prince*)
6. **Honoré de Balzac:** (*La Comédie humaine*)
7. **Guy de Maupassant:** (*Bel-Ami*, *Boule de Suif*)
8. **Emile Zola:** (*Germinal*, *Thérèse Raquin*)
9. **Michel Houellebecq:** (*Les Particules élémentaires*, *Soumission*)
10. **Patrick Modiano:** (*Rue des boutiques obscures*, *Dora Bruder*)


Cette liste n'est pas exhaustive et d'autres auteurs pourraient tout aussi bien y figurer selon les critères de sélection.



In [ ]:
import google.generativeai as genai

for m in genai.list_models():
    if "generateContent" in m.supported_generation_methods:
        print(m.name)

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.0-flash-exp
models/gemini-exp-1206
models/gemini-exp-1121
models/gemini-exp-1114
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/learnlm-1.5-pro-experimental


In [ ]:
from llm import get_azure_llm

llm = get_azure_llm()

In [ ]:
resp = llm.complete("Write a short, but joyous, ode to LlamaIndex")
print(resp)

Oh LlamaIndex, beacon bright,
In data's realm, you bring delight.
With wisdom vast and structure clear,
You guide us through the data sphere.

Your indexes, so deftly spun,
Unveil the secrets, one by one.
In fields of chaos, you bring order,
A trusted friend, a steadfast border.

Through queries deep and searches wide,
You stand as our unfailing guide.
With every byte and every node,
You lighten up our data load.

So here's to you, dear LlamaIndex,
A tool of joy, a friend complex.
In data's dance, you lead the way,
And brighten up our every day.


# tool selection

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
from llama_index.core.tools import FunctionTool


def add(x: int, y: int) -> int:
    """Adds two integers together."""
    return x + y


def mystery(x: int, y: int) -> int:
    """Mystery function that operates on top of two numbers."""
    return (x + y) * (x + y)


add_tool = FunctionTool.from_defaults(fn=add)
mystery_tool = FunctionTool.from_defaults(fn=mystery)

In [ ]:
from llm import get_azure_llm

llm = get_azure_llm()
response = llm.predict_and_call(
    [add_tool, mystery_tool],
    "Tell me the output of the mystery function on 2 and 9",
    verbose=True,
)
print(str(response))

=== Calling Function ===
Calling function: mystery with args: {"x": 2, "y": 9}
=== Function Output ===
121
121


# tool selection et teste avec Auteur

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
from llama_index.core.tools import FunctionTool

from mongo import Auteur


def add_auteur(nom: str) -> bool:
    """ajoute un auteur dans la base de données
    retourne un booléen indiquant si le nom etait pre existant dans la base de données

    nom: str : nom de l'auteur à ajouter, au format Prénom Nom (dans la mesure du possible, toujours insérer prénom puis nom, pas l'inverse)

    """

    auteur = Auteur(nom=nom)
    deja_present = auteur.exists()
    auteur.keep()
    return deja_present


addauteur_tool = FunctionTool.from_defaults(fn=add_auteur)

In [ ]:
from llm import get_azure_llm

llm = get_azure_llm()
response = llm.predict_and_call(
    [addauteur_tool],
    "Liste 10 romanciers français célébres. Et ajoute les dans la base",
    verbose=True,
)
print(str(response))

=== Calling Function ===
Calling function: add_auteur with args: {"nom": "Marcel Proust"}
=== Function Output ===
False
False


pas mal mais il ne continue pas, je dois faire ça avec un agent

# agent reasonning loop - openai

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
from llama_index.core.tools import FunctionTool

from mongo import Auteur


def add_auteur(nom: str) -> bool:
    """ajoute un auteur dans la base de données
    retourne un booléen indiquant si le nom etait pre existant dans la base de données

    nom: str : nom de l'auteur à ajouter, au format Prénom Nom (dans la mesure du possible, toujours insérer prénom puis nom, pas l'inverse)

    """

    auteur = Auteur(nom=nom)
    deja_present = auteur.exists()
    auteur.keep()
    return deja_present


addauteur_tool = FunctionTool.from_defaults(fn=add_auteur)

In [ ]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner
from llama_index.core import Settings
from llm import get_azure_llm


llm = get_azure_llm()

agent_worker = FunctionCallingAgentWorker.from_tools(
    [addauteur_tool], llm=llm, verbose=True
)
agent = AgentRunner(agent_worker)

In [ ]:
response = agent.query(
    "Liste 10 romanciers français célébres. Et ajoute les dans la base, et dis moi s'ils existaient deja"
)

Added user message to memory: Liste 10 romanciers français célébres. Et ajoute les dans la base, et dis moi s'ils existaient deja
=== LLM Response ===
Voici une liste de 10 romanciers français célèbres :

1. Victor Hugo
2. Marcel Proust
3. Gustave Flaubert
4. Émile Zola
5. Honoré de Balzac
6. Albert Camus
7. Stendhal (Henri Beyle)
8. Alexandre Dumas
9. Jules Verne
10. George Sand (Amantine Lucile Aurore Dupin)

Je vais maintenant les ajouter dans la base de données et vérifier s'ils existaient déjà.
=== Calling Function ===
Calling function: add_auteur with args: {"nom": "Victor Hugo"}
=== Function Output ===
True
=== Calling Function ===
Calling function: add_auteur with args: {"nom": "Marcel Proust"}
=== Function Output ===
True
=== Calling Function ===
Calling function: add_auteur with args: {"nom": "Gustave Flaubert"}
=== Function Output ===
False
=== Calling Function ===
Calling function: add_auteur with args: {"nom": "\u00c9mile Zola"}
=== Function Output ===
False
=== Calling Fu